In [1]:
import numpy as np
from time import time
from driver import Model

In [2]:
myip = Model("bitstream/cnn.bit")

In [3]:
# load test data, check accuracy against software implementation
data = np.int32(np.load("test_x.npy"))
gold = np.load("test_y.npy")

In [4]:
data.shape

(279, 75, 6)

In [ ]:
highest_false_positive_confidence = 0

for i in range(0, data.shape[0]):
    print("-"*40 + f"Data: {i}" + "-"*40)
    for j in range(0, data.shape[1]):
        print(data[i,j,:])
        predicted, softmax_confidence = myip.inference(data[i,j,:])
        gold_result = np.argmax(gold[i])
        print(f"raw outputs: {myip.raw_outputs}")
        print(f"input index: {j}")
        print(f"prediction: {predicted} vs GOLD: {gold_result}")
        print(f"confidence: {softmax_confidence * 100}%")
        print("\n")
        
        if predicted != np.argmax(gold[i]) and softmax_confidence > highest_false_positive_confidence:
            highest_false_positive_confidence = softmax_confidence
    myip.resetBuffer()

# if the confidence goes higher than this value then we do not need any more input
print(f"best threshold = {highest_false_positive_confidence}")

----------------------------------------Data: 0----------------------------------------
[-3449 -1981 -1039  1524    21   338]
raw outputs: [0. 0. 0. 0.]
input index: 0
prediction: 0 vs GOLD: 0
confidence: 25.0%


[-3450 -1939 -1043  1241    -4   216]
raw outputs: [0. 0. 0. 0.]
input index: 1
prediction: 0 vs GOLD: 0
confidence: 25.0%


[-3443 -1896 -1048   836    75   129]
raw outputs: [ 0.04008432 -0.00193747  0.08211122 -0.12262633]
input index: 2
prediction: 2 vs GOLD: 0
confidence: 27.077582478523254%


[-3422 -1850 -1044   469   219    54]
raw outputs: [ 0.0408049   0.01747395  0.07954316 -0.13803901]
input index: 3
prediction: 2 vs GOLD: 0
confidence: 26.98121964931488%


[-3406 -1808 -1033   113   261     8]
raw outputs: [ 0.04152548  0.03688537  0.07697511 -0.15345168]
input index: 4
prediction: 2 vs GOLD: 0
confidence: 26.88121795654297%


[-3395 -1778 -1012   -97   204    26]
raw outputs: [ 0.04224606  0.05629678  0.07440705 -0.16886435]
input index: 5
prediction: 2 vs GOLD: 

[-3485 -1726  -916   177   198    84]
raw outputs: [ 0.09124548  1.3762727  -0.10022066 -1.2169253 ]
input index: 73
prediction: 1 vs GOLD: 0
confidence: 63.296711444854736%


[-3496 -1712  -870    14    -9   164]
raw outputs: [ 0.09196605  1.395684   -0.10278869 -1.2323377 ]
input index: 74
prediction: 1 vs GOLD: 0
confidence: 63.807547092437744%


----------------------------------------Data: 1----------------------------------------
[-3434 -1793 -1009   374  -176  -135]
raw outputs: [0. 0. 0. 0.]
input index: 0
prediction: 0 vs GOLD: 0
confidence: 25.0%


[-3428 -1743 -1011   -24  -215  -132]
raw outputs: [0. 0. 0. 0.]
input index: 1
prediction: 0 vs GOLD: 0
confidence: 25.0%


[-3425 -1709 -1021  -414  -173  -138]
raw outputs: [ 0.04008432 -0.00193747  0.08211122 -0.12262633]
input index: 2
prediction: 2 vs GOLD: 0
confidence: 27.077582478523254%


[-3426 -1698 -1025  -752   -92  -140]
raw outputs: [ 0.0408049   0.01747395  0.07954316 -0.13803901]
input index: 3
prediction: 2 vs GO

raw outputs: [ 0.09124548  1.3762727  -0.10022066 -1.2169253 ]
input index: 73
prediction: 1 vs GOLD: 0
confidence: 63.296711444854736%


[-4589 -5690  -761 11359 -5163 -9931]
raw outputs: [ 0.09196605  1.395684   -0.10278869 -1.2323377 ]
input index: 74
prediction: 1 vs GOLD: 0
confidence: 63.807547092437744%


----------------------------------------Data: 2----------------------------------------
[-3407 -1885 -1056  -375   -39  -100]
raw outputs: [0. 0. 0. 0.]
input index: 0
prediction: 0 vs GOLD: 0
confidence: 25.0%


[-3403 -1892 -1037  -181    22    20]
raw outputs: [0. 0. 0. 0.]
input index: 1
prediction: 0 vs GOLD: 0
confidence: 25.0%


[-3394 -1892 -1009   -64   -46    84]
raw outputs: [ 0.04008432 -0.00193747  0.08211122 -0.12262633]
input index: 2
prediction: 2 vs GOLD: 0
confidence: 27.077582478523254%


[-3405 -1887  -985  -219  -125    25]
raw outputs: [ 0.0408049   0.01747395  0.07954316 -0.13803901]
input index: 3
prediction: 2 vs GOLD: 0
confidence: 26.98121964931488%



raw outputs: [ 0.0905249   1.3568614  -0.09765261 -1.2015127 ]
input index: 72
prediction: 1 vs GOLD: 0
confidence: 62.782251834869385%


[ 2673  -710 -1052  4168 -4138  -843]
raw outputs: [ 0.09124548  1.3762727  -0.10022066 -1.2169253 ]
input index: 73
prediction: 1 vs GOLD: 0
confidence: 63.296711444854736%


[ 3187  -441  -947  2624 -2801 -2307]
raw outputs: [ 0.09196605  1.395684   -0.10278869 -1.2323377 ]
input index: 74
prediction: 1 vs GOLD: 0
confidence: 63.807547092437744%


----------------------------------------Data: 3----------------------------------------
[-3411 -1883  -988  1782   678   613]
raw outputs: [0. 0. 0. 0.]
input index: 0
prediction: 0 vs GOLD: 0
confidence: 25.0%


[-3432 -1934  -947  1740   286   -46]
raw outputs: [0. 0. 0. 0.]
input index: 1
prediction: 0 vs GOLD: 0
confidence: 25.0%


[-3419 -1958  -934   768   781  -973]
raw outputs: [ 0.04008432 -0.00193747  0.08211122 -0.12262633]
input index: 2
prediction: 2 vs GOLD: 0
confidence: 27.077582478523254%

raw outputs: [ 0.0905249   1.3568614  -0.09765261 -1.2015127 ]
input index: 72
prediction: 1 vs GOLD: 0
confidence: 62.782251834869385%


[3132 -409 2311 -311  533  791]
raw outputs: [ 0.09124548  1.3762727  -0.10022066 -1.2169253 ]
input index: 73
prediction: 1 vs GOLD: 0
confidence: 63.296711444854736%


[3122 -445 2272 -385  653  853]
raw outputs: [ 0.09196605  1.395684   -0.10278869 -1.2323377 ]
input index: 74
prediction: 1 vs GOLD: 0
confidence: 63.807547092437744%


----------------------------------------Data: 4----------------------------------------
[ -4489  -5815   -893  10304  -5731 -11815]
raw outputs: [0. 0. 0. 0.]
input index: 0
prediction: 0 vs GOLD: 0
confidence: 25.0%


[ -4376  -5978  -1061   9411  -6457 -14531]
raw outputs: [0. 0. 0. 0.]
input index: 1
prediction: 0 vs GOLD: 0
confidence: 25.0%


[ -4221  -6093  -1265  10425  -6307 -15925]
raw outputs: [ 0.04008432 -0.00193747  0.08211122 -0.12262633]
input index: 2
prediction: 2 vs GOLD: 0
confidence: 27.0775824785

...

confidence: 48.4674870967865%


[ 5117  1913 -4850  -275 -2025  2094]
raw outputs: [ 0.07251044  0.87157637 -0.03345124 -0.8161967 ]
input index: 47
prediction: 1 vs GOLD: 2
confidence: 49.03859198093414%


[ 5208  1805 -5276 -1494 -2322  2210]
raw outputs: [ 0.073231    0.8909879  -0.0360193  -0.83160937]
input index: 48
prediction: 1 vs GOLD: 2
confidence: 49.60910975933075%


[ 5462  1913 -5749 -2366 -1930  2255]
raw outputs: [ 0.07395159  0.9103992  -0.03858736 -0.84702206]
input index: 49
prediction: 1 vs GOLD: 2
confidence: 50.17891526222229%


[ 5937  2358 -6389 -9709  -142  1320]
raw outputs: [ 0.07467216  0.9298105  -0.04115541 -0.8624346 ]
input index: 50
prediction: 1 vs GOLD: 2
confidence: 50.747841596603394%


[  6505   2369  -7195 -13237   3952   3164]
raw outputs: [ 0.07539275  0.949222   -0.04372347 -0.8778474 ]
input index: 51
prediction: 1 vs GOLD: 2
confidence: 51.31577253341675%


[ 6872  2338 -7968 -7353  8701  7439]
raw outputs: [ 0.07611333  0.96863335 -0.0462915

confidence: 48.4674870967865%


[ 1303   605  3389   443  2241 -1138]
raw outputs: [ 0.07251044  0.87157637 -0.03345124 -0.8161967 ]
input index: 47
prediction: 1 vs GOLD: 2
confidence: 49.03859198093414%


[ 1485   539  3157    75  2593 -1333]
raw outputs: [ 0.073231    0.8909879  -0.0360193  -0.83160937]
input index: 48
prediction: 1 vs GOLD: 2
confidence: 49.60910975933075%


[ 1612   506  2908   424  3492 -1464]
raw outputs: [ 0.07395159  0.9103992  -0.03858736 -0.84702206]
input index: 49
prediction: 1 vs GOLD: 2
confidence: 50.17891526222229%


[ 1630   492  2666    79  4905 -1499]
raw outputs: [ 0.07467216  0.9298105  -0.04115541 -0.8624346 ]
input index: 50
prediction: 1 vs GOLD: 2
confidence: 50.747841596603394%


[ 1604   431  2433 -1318  6059 -1841]
raw outputs: [ 0.07539275  0.949222   -0.04372347 -0.8778474 ]
input index: 51
prediction: 1 vs GOLD: 2
confidence: 51.31577253341675%


[ 1535   291  2202 -2765  6481 -2622]
raw outputs: [ 0.07611333  0.96863335 -0.04629152 -0.8

[-2990  -677  1214 -9988 14863  3425]
raw outputs: [ 0.07251044  0.87157637 -0.03345124 -0.8161967 ]
input index: 47
prediction: 1 vs GOLD: 2
confidence: 49.03859198093414%


[-3421  -936  1279 -9499 15035  4558]
raw outputs: [ 0.073231    0.8909879  -0.0360193  -0.83160937]
input index: 48
prediction: 1 vs GOLD: 2
confidence: 49.60910975933075%


[ -3817  -1144   1338 -10867  13562   4175]
raw outputs: [ 0.07395159  0.9103992  -0.03858736 -0.84702206]
input index: 49
prediction: 1 vs GOLD: 2
confidence: 50.17891526222229%


[ -4144  -1326   1288 -14133  11305   3118]
raw outputs: [ 0.07467216  0.9298105  -0.04115541 -0.8624346 ]
input index: 50
prediction: 1 vs GOLD: 2
confidence: 50.747841596603394%


[ -4422  -1594   1135 -15710  10073   3137]
raw outputs: [ 0.07539275  0.949222   -0.04372347 -0.8778474 ]
input index: 51
prediction: 1 vs GOLD: 2
confidence: 51.31577253341675%


[ -4711  -1937    977 -14851  10032   3950]
raw outputs: [ 0.07611333  0.96863335 -0.04629152 -0.89325994]

confidence: 49.03859198093414%


[-3912 -1915 -1128  3843  1321   594]
raw outputs: [ 0.073231    0.8909879  -0.0360193  -0.83160937]
input index: 48
prediction: 1 vs GOLD: 2
confidence: 49.60910975933075%


[-3840 -1722 -1195  3658  1279   624]
raw outputs: [ 0.07395159  0.9103992  -0.03858736 -0.84702206]
input index: 49
prediction: 1 vs GOLD: 2
confidence: 50.17891526222229%


[-3785 -1560 -1272  3208  1115   745]
raw outputs: [ 0.07467216  0.9298105  -0.04115541 -0.8624346 ]
input index: 50
prediction: 1 vs GOLD: 2
confidence: 50.747841596603394%


[-3749 -1436 -1344  2731   937   824]
raw outputs: [ 0.07539275  0.949222   -0.04372347 -0.8778474 ]
input index: 51
prediction: 1 vs GOLD: 2
confidence: 51.31577253341675%


[-3727 -1343 -1403  2268   697   771]
raw outputs: [ 0.07611333  0.96863335 -0.04629152 -0.89325994]
input index: 52
prediction: 1 vs GOLD: 2
confidence: 51.882559061050415%


[-3712 -1262 -1436  1871   529   626]
raw outputs: [ 0.0768339   0.9880448  -0.04885957 -0

raw outputs: [ 0.073231    0.8909879  -0.0360193  -0.83160937]
input index: 48
prediction: 1 vs GOLD: 3
confidence: 49.60910975933075%


[ -3024  -8031  -1517  -3834  -8110 -12217]
raw outputs: [ 0.07395159  0.9103992  -0.03858736 -0.84702206]
input index: 49
prediction: 1 vs GOLD: 3
confidence: 50.17891526222229%


[ -2713  -7657  -2435  -2169  -4447 -14359]
raw outputs: [ 0.07467216  0.9298105  -0.04115541 -0.8624346 ]
input index: 50
prediction: 1 vs GOLD: 3
confidence: 50.747841596603394%


[ -2464  -6993  -3035    -54   -511 -15460]
raw outputs: [ 0.07539275  0.949222   -0.04372347 -0.8778474 ]
input index: 51
prediction: 1 vs GOLD: 3
confidence: 51.31577253341675%


[ -2252  -6099  -3364    210   2312 -15643]
raw outputs: [ 0.07611333  0.96863335 -0.04629152 -0.89325994]
input index: 52
prediction: 1 vs GOLD: 3
confidence: 51.882559061050415%


[ -2054  -5145  -3471   -563   4778 -15203]
raw outputs: [ 0.0768339   0.9880448  -0.04885957 -0.9086726 ]
input index: 53
prediction: 1 